> 법률 상담 사례 크롤링

In [ ]:
#법률 상담 사례 case 추출

import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# 크롬 드라이버를 따로 다운받지 않아도 사용 가능하게 해주는 코드 :  cmd -> pip install 해주어야 함
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 크롤링할 url 선택
url = 'https://www.klac.or.kr/legalinfo/counsel.do'
driver.get(url)

# case 보관용 list
data = []

# >> 버튼 몇 번 클릭할 건지 계산
total_page = 1004
next_num = (total_page / 10) + 2

# for >> 버튼 클릭 반복
for page in range(1, next_num):

    # for > 버튼 클릭 반복
    for page in range(1, 11):

        # 데이터 있는 영역 크롤링
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        trs = soup.select('.table_TP03.M_table > tbody > tr')

        # 크롤링한 html 요소에서 원하는 데이터만 추출하는 코드
        for tr in trs:
            # case가 javascript 이 부분에 있으므로 이 부분의 데이터를 찾아 추출
            links = tr.find_all(href=lambda href: href and href.startswith("javascript:fn_inquire_detail('"))
            # links 데이터를 case number만 남도록 가공
            if links:
                match = re.search(r"case-\d+-\d+", links[0].get('href'))
                if match:
                    data.append({'상세주소': match.group(0)})
        
        # > 버튼 클릭
        page_button = driver.find_element(By.CSS_SELECTOR, f'#content > div.contents_doc > div > form > div.paging_wrap > a:nth-child({2 + page})')
        page_button.click()
        time.sleep(0.5)

    # >> 버튼 클릭
    next_button = driver.find_element(By.CSS_SELECTOR, '#content > div.contents_doc > div > form > div.paging_wrap > button.btn_page_next')
    next_button.click()
    time.sleep(0.5)

driver.close()

# list를 data로 변환하여 csv로 저장
# csv로 저장하지 않고 사용해도 되지만, 작업 중 중단 오류 발생 시 대처를 위한 방편
df = pd.DataFrame(data)
df.to_csv('D:/2. Git/TeamP/99. Sury/detail_link.csv')

In [1]:
# 법률 상담 사례 content 추출

import requests
from bs4 import BeautifulSoup
from tqdm import trange, notebook # 작업 진행 상황을 보기 위한 모듈, 없어도 무방
import pandas as pd

# 저장해둔 case 불러와 list로 저장
df = pd.read_csv('detail_link.csv')
addr = list(df['상세주소'])

# content 보관용 list 
data_list = []

# 크롤링 시 selenium 사용하면 시간이 오래 걸린다는 단점이 있으므로 bs4로 해결
# 작업 진행 상황이 필요 없을 경우 =  for case in addr: 
for case in notebook.tqdm(addr):
	# 개별 상세 주소 {case} 부분에 addr의 요소 삽입
	url = f'https://www.klac.or.kr/legalinfo/counselView.do?&folderId=000&caseId={case}&listNm=%EC%A0%84%EC%B2%B4&searchCnd=0&searchWrd=&scdFolderId='

	# 개별 상세 주소에 응답을 보내서 html 결과 값 받아오는 코드
	response = requests.get(url)
	
	# requests.get(url) 코드가 html을 받아오는 걸 성공 했을 경우
	if response.status_code == 200:
		soup = BeautifulSoup(response.text, 'html.parser')
		# 원하는 데이터 부분만 선택하는 코드
		a_element = soup.select_one('#print_page > div:nth-child(1) > dl > dd') 
		b_element = soup.select_one('#print_page > div:nth-child(2) > dl > dd')
		c_element = soup.select_one('#print_page > div:nth-child(3) > dl > dd')
		d_element = soup.select_one('#print_page > div:nth-child(4) > dl > dd')

		# a, b, c, d 부분의 데이터가 모두 있을 경우 data_list에 삽입
		if a_element and b_element and c_element and d_element:
			a = a_element.get_text().strip()
			b = b_element.get_text().strip()
			c = c_element.get_text().strip()
			d = d_element.get_text().strip()
			data_list.append({'구분': a, '제목': b, '질문': c, '답변': d})

		# 없을 경우 어느 부분이 없었는지 확인할 수 있게 case 와 함께 에러 문구 프린트
		else:
			print(f"Data not found for case : {case}")
	else:
		print(f"Failed to fetch data for case: {case}")

# list를 data로 변환하여 csv로 저장
df = pd.DataFrame(data_list)
df.to_csv('D:/2. Git/TeamP/99. Sury/detail_content.csv')

  0%|          | 0/10047 [00:00<?, ?it/s]

OSError: Cannot save file into a non-existent directory: 'D:\2. Git\TeamP\99. Sury'

In [2]:
df = pd.DataFrame(data_list)
df.to_csv('detail_content.csv')